# 4장. 메모리와 대화 관리

In [ ]:
!pip install langchain langchain-openai langgraph

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

---
## 코드 4-1 간단한 메모리: 대화 기록 유지하기

# 간단한 메모리: 대화 기록 유지하기

이 노트북에서는 **AI와의 대화 기록을 유지**하는 가장 기본적인 방법을 배웁니다.

## 왜 메모리가 필요할까요?

### AI의 기본 특성: 기억을 못함

```
┌────────────────────────────────────────────────────────────────────┐
│                    AI는 기본적으로 기억이 없다!                     │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   대화 1:                                                          │
│   사용자: "내 이름은 민혁이야"                                     │
│   AI: "안녕하세요 민혁님!"                                         │
│                                                                    │
│   대화 2 (새로운 요청):                                            │
│   사용자: "내 이름이 뭐야?"                                        │
│   AI: "죄송합니다, 저는 당신의 이름을 모릅니다" 😓                 │
│                                                                    │
│   → 각 요청은 완전히 독립적으로 처리됨!                            │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### 메모리의 역할

```
┌────────────────────────────────────────────────────────────────────┐
│                    메모리가 있으면                                  │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   대화 기록 저장:                                                  │
│   [                                                                │
│     {"user": "내 이름은 민혁이야"},                                │
│     {"ai": "안녕하세요 민혁님!"},                                   │
│     {"user": "내 이름이 뭐야?"}   ← 새 질문                        │
│   ]                                                                │
│                                                                    │
│   AI에게 전체 기록 전달 → AI가 맥락 파악                           │
│                                                                    │
│   AI: "당신의 이름은 민혁입니다!" ✅                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. placeholder를 사용한 메모리 구현

## placeholder란?

**placeholder**는 "자리 표시자"입니다. 대화 기록이 들어갈 자리를 미리 만들어둡니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    placeholder의 역할                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   프롬프트 템플릿:                                                 │
│   ┌────────────────────────────────────────┐                       │
│   │ System: 당신은 친절한 어시스턴트입니다  │                       │
│   │                                        │                       │
│   │ ▼▼▼ {messages} placeholder ▼▼▼        │  ← 여기에 대화 기록    │
│   │                                        │                       │
│   └────────────────────────────────────────┘                       │
│                                                                    │
│   실행 시:                                                         │
│   ┌────────────────────────────────────────┐                       │
│   │ System: 당신은 친절한 어시스턴트입니다  │                       │
│   │ Human: 나는 프로그래밍을 좋아해요      │                       │
│   │ AI: J'adore programmer.               │                       │
│   │ Human: 뭐라고 말했죠?                 │  ← 대화 기록이 채워짐  │
│   └────────────────────────────────────────┘                       │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

# 프롬프트 템플릿 생성
# 'placeholder'는 대화 기록이 들어갈 자리를 만들어줍니다
prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 친절한 어시스턴트입니다. 모든 질문에 최선을 다해 답하세요.'),
    ('placeholder', '{messages}'),  # 여기에 대화 기록이 들어감
])

# LLM 모델
model = ChatOllama(model='llama3.2')

# 체인 생성
chain = prompt | model

print("✅ 메모리 체인 생성 완료")

# 3. 대화 기록을 포함한 실행

이전 대화 내용을 함께 전달하여 AI가 맥락을 이해하게 합니다.

In [ ]:
# 대화 기록과 함께 실행
response = chain.invoke({
    'messages': [
        ('human', '다음 한국어 문장을 프랑스어로 번역하세요: 나는 프로그래밍을 좋아해요.'),
        ('ai', "J'adore programmer."),
        ('human', '뭐라고 말했죠?'),  # 이전 대화를 기억해야 답할 수 있음
    ],
})

print("=== AI 응답 ===")
print(response.content)

# 4. 메모리의 효과 확인

대화 기록 없이 같은 질문을 하면 어떻게 될까요?

In [ ]:
# 대화 기록 없이 실행 (메모리 없음)
response_no_memory = chain.invoke({
    'messages': [
        ('human', '뭐라고 말했죠?'),  # 맥락 없이 질문
    ],
})

print("=== 메모리 없이 응답 ===")
print(response_no_memory.content)
print("\n→ 맥락을 모르니까 뭘 물어보는지 모름!")

# 5. 대화 시뮬레이션

In [ ]:
# 대화 기록을 저장할 리스트
conversation_history = []

def chat(user_message):
    """
    사용자 메시지를 받아서 AI 응답을 반환하고
    대화 기록을 업데이트합니다.
    """
    # 사용자 메시지 추가
    conversation_history.append(('human', user_message))
    
    # AI 응답 생성 (전체 대화 기록 전달)
    response = chain.invoke({'messages': conversation_history})
    
    # AI 응답을 기록에 추가
    conversation_history.append(('ai', response.content))
    
    return response.content

print("✅ 대화 함수 준비 완료")

In [ ]:
# 대화 시뮬레이션
print("=== 대화 시작 ===")

# 첫 번째 대화
print("\n사용자: 안녕하세요! 제 이름은 민혁입니다.")
response1 = chat("안녕하세요! 제 이름은 민혁입니다.")
print(f"AI: {response1}")

# 두 번째 대화
print("\n사용자: 저는 파이썬 프로그래밍을 배우고 있어요.")
response2 = chat("저는 파이썬 프로그래밍을 배우고 있어요.")
print(f"AI: {response2}")

# 세 번째 대화 (이전 내용 기억 확인)
print("\n사용자: 제 이름이 뭐였죠?")
response3 = chat("제 이름이 뭐였죠?")
print(f"AI: {response3}")

In [ ]:
# 현재 대화 기록 확인
print("=== 현재 대화 기록 ===")
for i, (role, content) in enumerate(conversation_history):
    role_name = "사용자" if role == "human" else "AI"
    print(f"[{i+1}] {role_name}: {content[:50]}..." if len(content) > 50 else f"[{i+1}] {role_name}: {content}")

---

## 정리: 간단한 메모리

### 핵심 원리

```
┌────────────────────────────────────────────────────────────────────┐
│                    메모리의 핵심 원리                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   AI는 기본적으로 기억이 없다                                      │
│   → 우리가 대화 기록을 직접 관리해야 한다                          │
│   → 매번 요청할 때 전체 대화 기록을 함께 전달                      │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. placeholder로 대화 기록 자리 만들기
prompt = ChatPromptTemplate.from_messages([
    ('system', '시스템 메시지'),
    ('placeholder', '{messages}'),  # 대화 기록이 들어갈 자리
])

# 2. 대화 기록과 함께 실행
response = chain.invoke({
    'messages': [
        ('human', '첫 번째 질문'),
        ('ai', '첫 번째 답변'),
        ('human', '두 번째 질문'),
    ]
})
```

### 한계점

| 한계 | 설명 |
|------|------|
| **수동 관리** | 대화 기록을 직접 관리해야 함 |
| **토큰 제한** | 대화가 길어지면 토큰 한도 초과 |
| **영속성 없음** | 프로그램 종료 시 기록 사라짐 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 다음 단계

**LangGraph의 StateGraph**를 사용하여 더 체계적으로 대화 상태를 관리하는 방법을 배웁니다. (02-06번 노트북)

---
## 코드 4-2~4-6 LangGraph StateGraph로 챗봇 만들기

# LangGraph StateGraph로 챗봇 만들기

이 노트북에서는 **LangGraph**를 사용하여 **상태(State)를 관리하는 챗봇**을 만드는 방법을 배웁니다.

## LangGraph란?

**LangGraph**는 LangChain 팀이 만든 라이브러리로, **그래프 구조**로 AI 워크플로우를 만들 수 있게 해줍니다.

### 왜 LangGraph를 사용할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                LangChain vs LangGraph                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   LangChain (체인):                                                │
│   입력 → 처리1 → 처리2 → 처리3 → 출력                             │
│   (일직선으로 흐름)                                                │
│                                                                    │
│   LangGraph (그래프):                                              │
│                 ┌──→ 처리A ──┐                                     │
│   입력 → 분기점 ─┼──→ 처리B ──┼→ 합류점 → 출력                     │
│                 └──→ 처리C ──┘                                     │
│   (복잡한 흐름, 조건 분기, 반복 가능)                              │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### StateGraph의 구성 요소

```
┌────────────────────────────────────────────────────────────────────┐
│                  StateGraph의 3가지 요소                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ State (상태):                                                │
│      • 그래프 전체에서 공유되는 데이터                              │
│      • 예: 대화 기록, 사용자 정보 등                               │
│                                                                    │
│   2️⃣ Node (노드):                                                 │
│      • 실제 작업을 수행하는 함수                                   │
│      • 예: LLM 호출, 검색, 계산 등                                 │
│                                                                    │
│   3️⃣ Edge (엣지):                                                 │
│      • 노드 간의 연결                                              │
│      • 어떤 순서로 실행될지 결정                                   │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. State (상태) 정의하기

## TypedDict란?

**TypedDict**는 딕셔너리의 키와 값 타입을 미리 정의하는 방법입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    TypedDict 쉬운 설명                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 딕셔너리:                                                   │
│   data = {"name": "민혁", "age": 25}                              │
│   → 어떤 키가 있는지, 값 타입이 뭔지 모름                          │
│                                                                    │
│   TypedDict:                                                       │
│   class Person(TypedDict):                                         │
│       name: str                                                    │
│       age: int                                                     │
│   → "name은 문자열, age는 숫자"라고 명시                           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## add_messages란?

**add_messages**는 메시지를 **덮어쓰지 않고 추가**하는 방법입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    add_messages의 역할                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반적인 딕셔너리 업데이트:                                       │
│   state = {"messages": ["안녕"]}                                   │
│   state = {"messages": ["반가워"]}  # 덮어씀!                      │
│   결과: ["반가워"] ❌                                              │
│                                                                    │
│   add_messages 사용:                                               │
│   state = {"messages": ["안녕"]}                                   │
│   state = add_messages(["반가워"])  # 추가!                        │
│   결과: ["안녕", "반가워"] ✅                                      │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages

# 상태 정의
class State(TypedDict):
    """
    챗봇의 상태를 정의합니다.
    
    messages: 대화 기록 리스트
    - Annotated[list, add_messages]: 새 메시지를 추가하는 방식으로 업데이트
    """
    messages: Annotated[list, add_messages]

print("✅ State 정의 완료")
print("   - messages: 대화 기록을 저장하는 리스트")

# 3. Node (노드) 정의하기

노드는 실제 작업을 수행하는 함수입니다. 챗봇 노드는 LLM을 호출합니다.

In [ ]:
from langchain_ollama import ChatOllama

# LLM 모델
model = ChatOllama(model='llama3.2')

def chatbot(state: State):
    """
    챗봇 노드: LLM을 호출하여 응답을 생성합니다.
    
    입력: state (현재 상태, 대화 기록 포함)
    출력: {"messages": [응답]} (새 메시지를 추가)
    """
    # 현재까지의 대화 기록으로 LLM 호출
    answer = model.invoke(state['messages'])
    
    # 응답을 메시지 리스트에 추가
    return {'messages': [answer]}

print("✅ chatbot 노드 정의 완료")

# 4. Graph (그래프) 구성하기

노드와 엣지를 연결하여 그래프를 만듭니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    챗봇 그래프 구조                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│          ┌─────────┐      ┌─────────┐      ┌─────────┐            │
│          │  START  │ ───▶ │ chatbot │ ───▶ │   END   │            │
│          └─────────┘      └─────────┘      └─────────┘            │
│                                                                    │
│   START: 그래프 시작점                                             │
│   chatbot: LLM 호출 노드                                          │
│   END: 그래프 종료점                                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langgraph.graph import StateGraph, START, END

# 그래프 빌더 생성
builder = StateGraph(State)

# 노드 추가
# 첫 번째 인자: 노드 이름 (고유해야 함)
# 두 번째 인자: 실행할 함수
builder.add_node('chatbot', chatbot)

# 엣지 추가 (노드 간 연결)
builder.add_edge(START, 'chatbot')  # 시작 → 챗봇
builder.add_edge('chatbot', END)     # 챗봇 → 종료

# 그래프 컴파일
graph = builder.compile()

print("✅ 그래프 구성 완료")

# 5. 그래프 시각화

In [ ]:
# 그래프 구조를 Mermaid 형식으로 출력
print("=== 그래프 구조 (Mermaid) ===")
print(graph.get_graph().draw_mermaid())

# 6. 그래프 실행

## stream vs invoke

| 메서드 | 동작 | 사용 시나리오 |
|--------|------|---------------|
| **invoke** | 최종 결과만 반환 | 결과만 필요할 때 |
| **stream** | 단계별 결과 반환 | 진행 상황을 보고 싶을 때 |

In [ ]:
from langchain_core.messages import HumanMessage

# 입력 준비
input_message = {'messages': [HumanMessage(content='안녕하세요!')]}

print("=== 그래프 실행 (stream) ===")
print(f"입력: {input_message}\n")

# stream으로 실행 (단계별 출력)
for chunk in graph.stream(input_message):
    print(f"출력: {chunk}")

In [ ]:
# invoke로 실행 (최종 결과만)
result = graph.invoke(input_message)

print("=== 그래프 실행 (invoke) ===")
print(f"\n최종 응답: {result['messages'][-1].content}")

# 7. 여러 번 대화해보기

In [ ]:
# 여러 질문으로 테스트
test_messages = [
    "파이썬이란 무엇인가요?",
    "오늘 날씨가 어때요?",
    "간단한 파이썬 코드를 보여주세요.",
]

for msg in test_messages:
    print(f"\n{'='*50}")
    print(f"사용자: {msg}")
    
    result = graph.invoke({'messages': [HumanMessage(content=msg)]})
    
    print(f"\nAI: {result['messages'][-1].content[:200]}...")

---

## 정리: LangGraph StateGraph

### 핵심 구성요소

| 구성요소 | 역할 | 예시 |
|----------|------|------|
| **State** | 공유 데이터 정의 | `messages: list` |
| **Node** | 작업 수행 함수 | `chatbot()` |
| **Edge** | 노드 간 연결 | `START → chatbot → END` |

### 핵심 코드

```python
from langgraph.graph import StateGraph, START, END, add_messages

# 1. 상태 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 2. 노드 함수 정의
def chatbot(state: State):
    answer = model.invoke(state['messages'])
    return {'messages': [answer]}

# 3. 그래프 구성
builder = StateGraph(State)
builder.add_node('chatbot', chatbot)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

# 4. 컴파일 및 실행
graph = builder.compile()
result = graph.invoke({'messages': [...]})
```

### LangGraph의 장점

| 장점 | 설명 |
|------|------|
| **상태 관리** | State로 데이터 흐름 명확히 관리 |
| **확장성** | 노드 추가로 기능 확장 용이 |
| **시각화** | 그래프 구조 시각적 확인 가능 |
| **유연성** | 조건 분기, 반복 등 복잡한 흐름 구현 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 다음 단계

**MemorySaver**를 사용하여 대화 기록을 **영구 저장**하는 방법을 배웁니다. (07-10번 노트북)

---
## 코드 4-7~4-10 영속적 메모리: 대화 기록 저장하기

# 영속적 메모리: 대화 기록 저장하기

이 노트북에서는 **MemorySaver**를 사용하여 **대화 기록을 저장하고 복원**하는 방법을 배웁니다.

## 영속성(Persistence)이란?

```
┌────────────────────────────────────────────────────────────────────┐
│               영속성 없음 vs 영속성 있음                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   영속성 없음 (이전 노트북):                                        │
│   • 프로그램 종료 → 대화 기록 사라짐 💨                            │
│   • 다시 시작하면 처음부터                                         │
│                                                                    │
│   영속성 있음 (이 노트북):                                          │
│   • 프로그램 종료 → 대화 기록 저장됨 💾                            │
│   • 다시 시작해도 이전 대화 기억                                   │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## Thread ID란?

**Thread ID**는 대화를 구분하는 고유 식별자입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    Thread ID의 역할                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   카카오톡 채팅방처럼 생각하세요!                                   │
│                                                                    │
│   Thread 1 (친구A와 대화):                                         │
│   ├─ "오늘 뭐해?"                                                  │
│   ├─ "영화 보러 가자"                                              │
│   └─ "좋아!"                                                       │
│                                                                    │
│   Thread 2 (친구B와 대화):                                         │
│   ├─ "숙제 했어?"                                                  │
│   └─ "아직..."                                                     │
│                                                                    │
│   → 각 Thread는 독립적인 대화 기록을 가짐                          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 그래프 구성 (이전 노트북과 동일)

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END, add_messages

# 상태 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 그래프 빌더
builder = StateGraph(State)

# LLM 모델
model = ChatOllama(model='llama3.2')

# 챗봇 노드
def chatbot(state: State):
    answer = model.invoke(state['messages'])
    return {'messages': [answer]}

# 노드와 엣지 추가
builder.add_node('chatbot', chatbot)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

print("✅ 그래프 구성 완료")

# 3. MemorySaver로 영속성 추가

**MemorySaver**는 대화 상태를 메모리에 저장합니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    MemorySaver의 역할                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   그래프 실행 시:                                                  │
│   1. 입력 메시지 처리                                              │
│   2. 노드 실행                                                     │
│   3. 결과를 MemorySaver에 저장 ← 여기가 핵심!                      │
│                                                                    │
│   다음 실행 시:                                                    │
│   1. MemorySaver에서 이전 상태 불러오기                            │
│   2. 새 메시지와 합쳐서 처리                                       │
│   3. 결과 다시 저장                                                │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# MemorySaver 생성
memory = MemorySaver()

# checkpointer 옵션과 함께 그래프 컴파일
graph = builder.compile(checkpointer=memory)

print("✅ 영속성이 추가된 그래프 컴파일 완료")

# 4. Thread ID를 사용한 대화

같은 Thread ID를 사용하면 이전 대화를 기억합니다.

In [ ]:
# Thread 설정 (대화방 ID라고 생각하세요)
thread1 = {'configurable': {'thread_id': '1'}}

print("=== Thread 1에서 대화 ===")

# 첫 번째 대화
print("\n[대화 1]")
print("사용자: 안녕하세요, 저는 민혁입니다!")
result1 = graph.invoke(
    {'messages': [HumanMessage(content='안녕하세요, 저는 민혁입니다!')]},
    thread1  # Thread ID 전달
)
print(f"AI: {result1['messages'][-1].content}")

In [ ]:
# 두 번째 대화 (같은 Thread)
print("\n[대화 2]")
print("사용자: 제 이름이 뭐죠?")
result2 = graph.invoke(
    {'messages': [HumanMessage(content='제 이름이 뭐죠?')]},
    thread1  # 같은 Thread ID
)
print(f"AI: {result2['messages'][-1].content}")

print("\n✅ 이전 대화를 기억합니다!")

# 5. 상태 확인하기

현재 저장된 대화 상태를 확인할 수 있습니다.

In [ ]:
# 현재 상태 확인
state = graph.get_state(thread1)

print("=== 현재 Thread 1의 상태 ===")
print(f"\n메시지 수: {len(state.values['messages'])}개\n")

for i, msg in enumerate(state.values['messages']):
    role = "사용자" if msg.type == "human" else "AI"
    content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"[{i+1}] {role}: {content}")

# 6. 상태 업데이트하기

대화 기록을 수동으로 수정할 수도 있습니다.

In [ ]:
# 상태 업데이트 (새 메시지 추가)
graph.update_state(
    thread1,
    {'messages': [HumanMessage(content='저는 LLM이 좋아요!')]}
)

print("✅ 상태 업데이트 완료")

# 업데이트된 상태 확인
state = graph.get_state(thread1)
print(f"\n업데이트 후 메시지 수: {len(state.values['messages'])}개")

# 7. 다른 Thread에서 대화

Thread ID가 다르면 완전히 독립적인 대화입니다.

In [ ]:
# 다른 Thread
thread2 = {'configurable': {'thread_id': '2'}}

print("=== Thread 2에서 대화 (새로운 대화) ===")

print("\n사용자: 제 이름이 뭐죠?")
result = graph.invoke(
    {'messages': [HumanMessage(content='제 이름이 뭐죠?')]},
    thread2  # 다른 Thread ID
)
print(f"AI: {result['messages'][-1].content}")

print("\n→ Thread 2는 Thread 1의 대화를 모릅니다!")

In [ ]:
# Thread 1로 돌아가면 여전히 기억함
print("=== 다시 Thread 1로 돌아가기 ===")

print("\n사용자: 제가 좋아하는 게 뭐라고 했죠?")
result = graph.invoke(
    {'messages': [HumanMessage(content='제가 좋아하는 게 뭐라고 했죠?')]},
    thread1  # 다시 Thread 1
)
print(f"AI: {result['messages'][-1].content}")

---

## 정리: 영속적 메모리

### 핵심 개념

| 개념 | 설명 | 비유 |
|------|------|------|
| **MemorySaver** | 상태를 저장하는 도구 | 저장 버튼 |
| **Thread ID** | 대화를 구분하는 ID | 채팅방 번호 |
| **checkpointer** | 저장 도구 연결 옵션 | 자동 저장 설정 |
| **get_state** | 저장된 상태 조회 | 기록 보기 |
| **update_state** | 상태 수동 수정 | 기록 편집 |

### 핵심 코드

```python
from langgraph.checkpoint.memory import MemorySaver

# 1. MemorySaver 생성
memory = MemorySaver()

# 2. checkpointer와 함께 컴파일
graph = builder.compile(checkpointer=memory)

# 3. Thread ID와 함께 실행
thread = {'configurable': {'thread_id': '1'}}
result = graph.invoke({'messages': [...]}, thread)

# 4. 상태 조회
state = graph.get_state(thread)

# 5. 상태 업데이트
graph.update_state(thread, {'messages': [...]})
```

### 주의사항

| 항목 | MemorySaver 특성 |
|------|------------------|
| **저장 위치** | 메모리 (RAM) |
| **프로그램 종료 시** | 데이터 사라짐 |
| **영구 저장 필요 시** | SQLite, PostgreSQL 등 사용 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
model = ChatOllama(model='llama3.2')
```

## 다음 단계

대화가 길어질 때 **메시지를 자르는(Trim)** 방법을 배웁니다. (11번 노트북)

---
## 코드 4-11 메시지 자르기 (Trim Messages)

# 메시지 자르기 (Trim Messages)

이 노트북에서는 **대화가 길어질 때 메시지를 자르는** 방법을 배웁니다.

## 왜 메시지를 잘라야 할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    토큰 제한 문제                                  │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   LLM의 한계:                                                      │
│   • 한 번에 처리할 수 있는 토큰 수가 제한됨                         │
│   • GPT-4: ~128K 토큰, Llama: ~8K 토큰                             │
│   • 대화가 길어지면 한도 초과!                                      │
│                                                                    │
│   해결책:                                                          │
│   • 오래된 메시지 삭제                                              │
│   • 최근 메시지만 유지                                              │
│   • 시스템 메시지는 항상 유지                                       │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## trim_messages의 전략

```
┌────────────────────────────────────────────────────────────────────┐
│                 trim_messages 전략 비교                            │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   strategy='last' (기본):                                          │
│   [시스템] [메시지1] [메시지2] [메시지3] [메시지4] [메시지5]        │
│              ↓         ↓                                          │
│            삭제       삭제                                         │
│   결과: [시스템] [메시지3] [메시지4] [메시지5]                      │
│   → 최근 메시지 유지                                               │
│                                                                    │
│   strategy='first':                                                │
│   [시스템] [메시지1] [메시지2] [메시지3] [메시지4] [메시지5]        │
│                                         ↓         ↓               │
│                                       삭제       삭제              │
│   결과: [시스템] [메시지1] [메시지2] [메시지3]                      │
│   → 초기 메시지 유지                                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 샘플 메시지 준비

긴 대화 기록을 만들어봅시다.

In [ ]:
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
)

# 긴 대화 기록 (11개 메시지)
messages = [
    SystemMessage(content='당신은 친절한 어시스턴트입니다.'),
    HumanMessage(content='안녕하세요! 나는 민혁입니다.'),
    AIMessage(content='안녕하세요!'),
    HumanMessage(content='바닐라 아이스크림을 좋아해요.'),
    AIMessage(content='좋네요!'),
    HumanMessage(content='2 + 2는 얼마죠?'),
    AIMessage(content='4입니다.'),
    HumanMessage(content='고마워요.'),
    AIMessage(content='천만에요!'),
    HumanMessage(content='즐거운가요?'),
    AIMessage(content='예!'),
]

print(f"총 메시지 수: {len(messages)}개\n")
print("=== 전체 대화 기록 ===")
for i, msg in enumerate(messages):
    role = msg.type
    print(f"[{i+1}] {role}: {msg.content}")

# 3. trim_messages 사용하기

## 주요 옵션 설명

| 옵션 | 설명 | 예시 |
|------|------|------|
| **max_tokens** | 최대 토큰 수 | 65 |
| **strategy** | 자르기 전략 | 'last' (최근 유지) |
| **token_counter** | 토큰 계산 모델 | LLM 모델 |
| **include_system** | 시스템 메시지 포함 | True |
| **allow_partial** | 메시지 부분 자르기 허용 | False |
| **start_on** | 시작 메시지 유형 | 'human' |

In [ ]:
from langchain_core.messages import trim_messages
from langchain_ollama import ChatOllama

# LLM 모델 (토큰 계산용)
model = ChatOllama(model='llama3.2')

# trimmer 설정
trimmer = trim_messages(
    max_tokens=65,              # 최대 65 토큰만 유지
    strategy='last',            # 최근 메시지 우선 유지
    token_counter=model,        # 토큰 계산에 사용할 모델
    include_system=True,        # 시스템 메시지는 항상 포함
    allow_partial=False,        # 메시지를 중간에 자르지 않음
    start_on='human',           # human 메시지로 시작
)

print("✅ trimmer 설정 완료")
print("   - 최대 65 토큰")
print("   - 최근 메시지 유지 전략")
print("   - 시스템 메시지 항상 포함")

# 4. 메시지 자르기 실행

In [ ]:
# trim 적용
trimmed = trimmer.invoke(messages)

print(f"원본 메시지 수: {len(messages)}개")
print(f"자른 후 메시지 수: {len(trimmed)}개\n")

print("=== 자른 후 대화 기록 ===")
for i, msg in enumerate(trimmed):
    role = msg.type
    print(f"[{i+1}] {role}: {msg.content}")

# 5. 다양한 설정 테스트

In [ ]:
# 더 많은 토큰 허용
trimmer_large = trim_messages(
    max_tokens=100,
    strategy='last',
    token_counter=model,
    include_system=True,
)

trimmed_large = trimmer_large.invoke(messages)

print("=== max_tokens=100 ===")
print(f"메시지 수: {len(trimmed_large)}개")
for msg in trimmed_large:
    print(f"  {msg.type}: {msg.content}")

In [ ]:
# strategy='first' 테스트 (초기 메시지 유지)
trimmer_first = trim_messages(
    max_tokens=65,
    strategy='first',           # 초기 메시지 유지
    token_counter=model,
    include_system=True,
)

trimmed_first = trimmer_first.invoke(messages)

print("\n=== strategy='first' ===")
print(f"메시지 수: {len(trimmed_first)}개")
for msg in trimmed_first:
    print(f"  {msg.type}: {msg.content}")

# 6. 체인에서 trimmer 사용하기

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# trimmer를 체인의 일부로 사용
prompt = ChatPromptTemplate.from_messages([
    ('placeholder', '{messages}'),
])

# 체인: trimmer → prompt → model
chain = trimmer | prompt | model

print("✅ trimmer가 포함된 체인 생성 완료")

In [ ]:
# 체인 실행
result = chain.invoke(messages)

print("=== 체인 실행 결과 ===")
print(f"AI 응답: {result.content}")

---

## 정리: 메시지 자르기 (trim_messages)

### 주요 옵션

| 옵션 | 기본값 | 설명 |
|------|--------|------|
| **max_tokens** | 필수 | 유지할 최대 토큰 수 |
| **strategy** | 'last' | 'last': 최근 유지, 'first': 초기 유지 |
| **token_counter** | 필수 | 토큰 계산에 사용할 모델/함수 |
| **include_system** | False | 시스템 메시지 항상 포함 여부 |
| **allow_partial** | False | 메시지 중간 자르기 허용 |
| **start_on** | None | 시작해야 할 메시지 유형 |

### 핵심 코드

```python
from langchain_core.messages import trim_messages

# 1. trimmer 생성
trimmer = trim_messages(
    max_tokens=100,
    strategy='last',
    token_counter=model,
    include_system=True,
)

# 2. 메시지 자르기
trimmed = trimmer.invoke(messages)

# 3. 체인에서 사용
chain = trimmer | prompt | model
```

### 전략 선택 가이드

| 상황 | 추천 전략 |
|------|----------|
| 일반 대화 | `strategy='last'` (최근 맥락 중요) |
| 문서 요약 | `strategy='first'` (초기 내용 중요) |
| 시스템 지시 중요 | `include_system=True` |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
from langchain_openai import ChatOpenAI
token_counter = ChatOpenAI(model='gpt-4o-mini')

# 변경
from langchain_ollama import ChatOllama
token_counter = ChatOllama(model='llama3.2')
```

## 다음 단계

**메시지를 유형별로 필터링**하는 방법을 배웁니다. (12-14번 노트북)

---
## 코드 4-12~4-14 메시지 필터링 (Filter Messages)

# 메시지 필터링 (Filter Messages)

이 노트북에서는 **특정 조건에 맞는 메시지만 선택**하는 방법을 배웁니다.

## 필터링이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    필터링의 다양한 기준                            │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   필터링 기준:                                                     │
│                                                                    │
│   1️⃣ 유형(Type)으로 필터링                                        │
│      • human 메시지만 추출                                         │
│      • AI 메시지만 추출                                            │
│      • system 메시지 제외                                          │
│                                                                    │
│   2️⃣ 이름(Name)으로 필터링                                        │
│      • 특정 사용자의 메시지만                                      │
│      • 예시 메시지 제외                                            │
│                                                                    │
│   3️⃣ ID로 필터링                                                  │
│      • 특정 ID의 메시지 포함/제외                                  │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 언제 필터링이 필요할까요?

| 상황 | 필터링 방법 |
|------|------------|
| Few-shot 예시 제거 | 예시 이름으로 제외 |
| 사용자 발언만 분석 | human 유형만 포함 |
| 특정 대화 제외 | ID로 제외 |

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 샘플 메시지 준비

다양한 속성을 가진 메시지들을 준비합니다.

In [ ]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

# 다양한 속성을 가진 메시지들
messages = [
    SystemMessage(content='당신은 친절한 어시스턴트입니다.', id='1'),
    HumanMessage(content='예시 입력', id='2', name='example_user'),      # 예시 메시지
    AIMessage(content='예시 출력', id='3', name='example_assistant'),    # 예시 메시지
    HumanMessage(content='실제 입력', id='4', name='bob'),               # 실제 사용자
    AIMessage(content='실제 출력', id='5', name='alice'),                # 실제 AI
]

print("=== 전체 메시지 ===")
for msg in messages:
    name = f" (name={msg.name})" if hasattr(msg, 'name') and msg.name else ""
    print(f"[{msg.id}] {msg.type}{name}: {msg.content}")

# 3. 유형(Type)으로 필터링

In [ ]:
from langchain_core.messages import filter_messages

# Human 메시지만 추출
human_messages = filter_messages(messages, include_types='human')

print("=== Human 메시지만 ===")
for msg in human_messages:
    print(f"[{msg.id}] {msg.content}")

In [ ]:
# AI 메시지만 추출
ai_messages = filter_messages(messages, include_types='ai')

print("=== AI 메시지만 ===")
for msg in ai_messages:
    print(f"[{msg.id}] {msg.content}")

In [ ]:
# 여러 유형 포함 (Human + AI)
human_ai_messages = filter_messages(messages, include_types=['human', 'ai'])

print("=== Human + AI 메시지 (System 제외) ===")
for msg in human_ai_messages:
    print(f"[{msg.id}] {msg.type}: {msg.content}")

# 4. 이름(Name)으로 필터링

In [ ]:
# 특정 이름 제외 (예시 메시지 제외)
excluded_examples = filter_messages(
    messages, 
    exclude_names=['example_user', 'example_assistant']
)

print("=== 예시 메시지 제외 ===")
for msg in excluded_examples:
    name = f" (name={msg.name})" if hasattr(msg, 'name') and msg.name else ""
    print(f"[{msg.id}] {msg.type}{name}: {msg.content}")

In [ ]:
# 특정 이름만 포함
bob_messages = filter_messages(
    messages,
    include_names=['bob']
)

print("=== bob의 메시지만 ===")
for msg in bob_messages:
    print(f"[{msg.id}] {msg.content}")

# 5. ID로 필터링

In [ ]:
# 특정 ID 제외
without_id3 = filter_messages(
    messages,
    exclude_ids=['3']  # ID가 '3'인 메시지 제외
)

print("=== ID '3' 제외 ===")
for msg in without_id3:
    print(f"[{msg.id}] {msg.type}: {msg.content}")

# 6. 복합 조건 필터링

In [ ]:
# 유형 + ID 조합 필터링
filtered = filter_messages(
    messages,
    include_types=['human', 'ai'],  # human과 ai만
    exclude_ids=['3']               # ID '3' 제외
)

print("=== human/ai 유형 중 ID '3' 제외 ===")
for msg in filtered:
    print(f"[{msg.id}] {msg.type}: {msg.content}")

# 7. 체인에서 필터 사용하기

In [ ]:
from langchain_ollama import ChatOllama

# LLM 모델
model = ChatOllama(model='llama3.2')

# 예시 메시지를 제외하는 필터
filter_ = filter_messages(exclude_names=['example_user', 'example_assistant'])

# 체인: 필터 → 모델
chain = filter_ | model

print("✅ 필터가 포함된 체인 생성 완료")

In [ ]:
# 체인 실행
# 전체 메시지를 전달하지만, 필터가 예시 메시지를 제외함
result = chain.invoke(messages)

print("=== 필터 체인 실행 결과 ===")
print(f"입력: 전체 {len(messages)}개 메시지")
print(f"필터 후: 예시 메시지 제외")
print(f"\nAI 응답: {result.content}")

---

## 정리: 메시지 필터링 (filter_messages)

### 주요 옵션

| 옵션 | 설명 | 예시 |
|------|------|------|
| **include_types** | 포함할 메시지 유형 | `'human'`, `['human', 'ai']` |
| **exclude_types** | 제외할 메시지 유형 | `'system'` |
| **include_names** | 포함할 이름 | `['bob']` |
| **exclude_names** | 제외할 이름 | `['example_user']` |
| **include_ids** | 포함할 ID | `['1', '2']` |
| **exclude_ids** | 제외할 ID | `['3']` |

### 핵심 코드

```python
from langchain_core.messages import filter_messages

# 1. 유형으로 필터링
human_msgs = filter_messages(messages, include_types='human')

# 2. 이름으로 필터링
filtered = filter_messages(messages, exclude_names=['example'])

# 3. 복합 조건
filtered = filter_messages(
    messages,
    include_types=['human', 'ai'],
    exclude_ids=['3']
)

# 4. 체인에서 사용
filter_ = filter_messages(exclude_names=[...])
chain = filter_ | model
```

### 활용 사례

| 사례 | 필터링 방법 |
|------|------------|
| Few-shot 예시 제거 | `exclude_names=['example_*']` |
| 사용자 발언만 분석 | `include_types='human'` |
| 시스템 메시지 제외 | `exclude_types='system'` |
| 특정 대화 제외 | `exclude_ids=['id']` |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
model = ChatOllama(model='llama3.2')
```

## 다음 단계

**연속된 같은 유형의 메시지를 병합**하는 방법을 배웁니다. (15-16번 노트북)

---
## 코드 4-15~4-16 메시지 병합 (Merge Messages)

# 메시지 병합 (Merge Messages)

이 노트북에서는 **연속된 같은 유형의 메시지를 하나로 병합**하는 방법을 배웁니다.

## 왜 메시지를 병합할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    메시지 병합이 필요한 상황                        │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   문제 상황:                                                       │
│   [System] 당신은 친절한 어시스턴트입니다.                         │
│   [System] 항상 농담으로 대답하세요.      ← 연속된 System 메시지   │
│   [Human] 어떤 피자가 맛있나요?                                    │
│   [Human] 어떤 햄버거가 맛있나요?         ← 연속된 Human 메시지    │
│                                                                    │
│   일부 LLM은 연속된 같은 유형의 메시지를 처리 못함!                │
│                                                                    │
│   해결:                                                            │
│   [System] 당신은 친절한 어시스턴트입니다. 항상 농담으로 대답하세요.│
│   [Human] 어떤 피자가 맛있나요? 어떤 햄버거가 맛있나요?            │
│                                                                    │
│   → 같은 유형의 연속 메시지를 하나로 합침!                         │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 연속된 메시지 준비

In [ ]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

# 연속된 같은 유형의 메시지가 있는 대화
messages = [
    SystemMessage(content='당신은 친절한 어시스턴트입니다.'),
    SystemMessage(content='항상 농담으로 대답하세요.'),  # 연속된 System
    HumanMessage(
        content=[{'type': 'text', 'text': '어떤 피자가 제일 맛있나요?'}]
    ),
    HumanMessage(content='어떤 햄버거가 가장 맛있나요?'),  # 연속된 Human
    AIMessage(content='나는 항상 너만 "고르곤졸라"'),
    AIMessage(content='너가 "버거" 싶어'),  # 연속된 AI
]

print(f"=== 병합 전 ({len(messages)}개 메시지) ===")
for i, msg in enumerate(messages):
    content = str(msg.content)[:40]
    print(f"[{i+1}] {msg.type}: {content}..." if len(str(msg.content)) > 40 else f"[{i+1}] {msg.type}: {content}")

# 3. merge_message_runs 사용하기

In [ ]:
from langchain_core.messages import merge_message_runs

# 연속된 메시지 병합
merged = merge_message_runs(messages)

print(f"=== 병합 후 ({len(merged)}개 메시지) ===")
for i, msg in enumerate(merged):
    print(f"\n[{i+1}] {msg.type}:")
    print(f"    {msg.content}")

# 4. 병합 결과 자세히 보기

In [ ]:
print("=== 병합 전후 비교 ===")
print("\n[병합 전]")
print(f"  System 메시지: {len([m for m in messages if m.type == 'system'])}개")
print(f"  Human 메시지: {len([m for m in messages if m.type == 'human'])}개")
print(f"  AI 메시지: {len([m for m in messages if m.type == 'ai'])}개")

print("\n[병합 후]")
print(f"  System 메시지: {len([m for m in merged if m.type == 'system'])}개")
print(f"  Human 메시지: {len([m for m in merged if m.type == 'human'])}개")
print(f"  AI 메시지: {len([m for m in merged if m.type == 'ai'])}개")

# 5. 체인에서 merger 사용하기

In [ ]:
from langchain_ollama import ChatOllama

# LLM 모델
model = ChatOllama(model='llama3.2')

# merger 생성 (체인에서 사용할 때)
merger = merge_message_runs()

# 체인: 병합 → 모델
chain = merger | model

print("✅ merger가 포함된 체인 생성 완료")

In [ ]:
# 체인 실행
result = chain.invoke(messages)

print("=== 병합 체인 실행 결과 ===")
print(f"입력: {len(messages)}개 메시지")
print(f"병합 후: {len(merged)}개 메시지")
print(f"\nAI 응답: {result.content}")

# 6. 다양한 상황 테스트

In [ ]:
# 연속되지 않은 메시지는 병합되지 않음
alternating_messages = [
    HumanMessage(content='안녕하세요'),
    AIMessage(content='반가워요'),
    HumanMessage(content='오늘 날씨 어때요?'),
    AIMessage(content='좋아요!'),
]

merged_alternating = merge_message_runs(alternating_messages)

print("=== 번갈아 나오는 메시지 (병합 안됨) ===")
print(f"병합 전: {len(alternating_messages)}개")
print(f"병합 후: {len(merged_alternating)}개")
print("\n→ 연속되지 않아서 병합 안됨!")

In [ ]:
# 여러 개의 연속 메시지 테스트
many_consecutive = [
    HumanMessage(content='첫 번째 질문'),
    HumanMessage(content='두 번째 질문'),
    HumanMessage(content='세 번째 질문'),
    HumanMessage(content='네 번째 질문'),
]

merged_many = merge_message_runs(many_consecutive)

print("=== 4개 연속 Human 메시지 ===")
print(f"병합 전: {len(many_consecutive)}개")
print(f"병합 후: {len(merged_many)}개")
print(f"\n병합된 내용:\n{merged_many[0].content}")

---

## 정리: 메시지 병합 (merge_message_runs)

### 핵심 동작

```
┌────────────────────────────────────────────────────────────────────┐
│                merge_message_runs 동작 원리                        │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   [System] A                                                       │
│   [System] B    →  [System] A + B                                 │
│                                                                    │
│   [Human] 질문1                                                    │
│   [Human] 질문2  →  [Human] 질문1 + 질문2                          │
│                                                                    │
│   [AI] 답변1                                                       │
│   [AI] 답변2     →  [AI] 답변1 + 답변2                             │
│                                                                    │
│   → 같은 유형이 연속되면 내용을 합침                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
from langchain_core.messages import merge_message_runs

# 1. 직접 사용
merged = merge_message_runs(messages)

# 2. 체인에서 사용
merger = merge_message_runs()
chain = merger | model
result = chain.invoke(messages)
```

### 언제 사용할까요?

| 상황 | 사용 여부 |
|------|----------|
| 연속된 시스템 지시 통합 | ✅ 사용 |
| 여러 사용자 입력 통합 | ✅ 사용 |
| API 제한 우회 | ✅ 사용 |
| 번갈아 나오는 메시지 | ❌ 불필요 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
model = ChatOllama(model='llama3.2')
```

## ch04 요약: 메모리 관리 기법

| 기법 | 용도 | 핵심 함수/클래스 |
|------|------|------------------|
| **Simple Memory** | 대화 기록 유지 | `placeholder` |
| **StateGraph** | 상태 기반 챗봇 | `StateGraph`, `add_messages` |
| **Persistent Memory** | 대화 저장/복원 | `MemorySaver`, Thread ID |
| **Trim Messages** | 토큰 제한 대응 | `trim_messages` |
| **Filter Messages** | 조건별 필터링 | `filter_messages` |
| **Merge Messages** | 연속 메시지 병합 | `merge_message_runs` |